In [1]:
class Problem:
    '''
    Abstract base class for problem formulation.
    It declares the expected methods to be used by a search algorithm.
    All the methods declared are just placeholders that throw errors if not overriden by child "concrete" classes!
    '''
    
    def __init__(self):
        '''Constructor that initializes the problem. Typically used to setup the initial state and, if applicable, the goal state.'''
        self.init_state = None
    
    def actions(self, state):
        '''Returns an iterable with the applicable actions to the given state.'''
        raise NotImplementedError
    
    def result(self, state, action):
        '''Returns the resulting state from applying the given action to the given state.'''
        raise NotImplementedError
    
    def goal_test(self, state):
        '''Returns whether or not the given state is a goal state.'''
        raise NotImplementedError
    
    def step_cost(self, state, action):
        '''Returns the step cost of applying the given action to the given state.'''
        raise NotImplementedError

In [6]:
class Node:
    '''Node data structure for search space bookkeeping.'''
    
    def __init__(self, state, parent, action, path_cost):
        '''Constructor for the node state with the required parameters.'''
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost

    @classmethod
    def root(cls, init_state):
        '''Factory method to create the root node.'''
        return cls(init_state, None, None, 0)

    @classmethod
    def child(cls, problem, parent, action):
        '''Factory method to create a child node.'''
        return cls(
            problem.result(parent.state, action),
            parent,
            action,
            parent.path_cost + problem.step_cost(parent.state, action))

def solution(node):
    '''A method to extract the sequence of actions representing the solution from the goal node.'''
    actions = []
    cost = node.path_cost
    while node.parent is not None:
        actions.append(node.action)
        node = node.parent
    actions.reverse()
    return actions, cost

In [7]:
from shutil import get_terminal_size
terminal_width, _ = get_terminal_size()

_visualizers = {}

def _default_visualizer(_, state):
    '''Generic visualizer for unknown problems.'''
    print(state)

class Visualizer:
    '''Visualization and printing functionality encapsulation.'''

    def __init__(self, problem):
        '''Constructor with the problem to visualize.'''
        self.problem = problem
        self.counter = 0
    
    def visualize(self, frontier):
        '''Visualizes the frontier at every step.'''
        self.counter += 1
        print(f'Frontier at step {self.counter}')
        for node in frontier:
            print()
            _visualizers.get(type(self.problem), _default_visualizer)(self.problem, node.state)
        print('-' * terminal_width)

In [8]:
class SlidingPuzzle3x3(Problem):
    '''3x3 Sliding Puzzle problem formulation.'''

    def __init__(self, init_state, goal_state):
        assert init_state.count(' ') == 1
        assert goal_state.count(' ') == 1
        self.init_state = tuple(init_state)
        self._goal_state = tuple(goal_state)
        self._action_values = {'up': -3, 'down': +3, 'left': -1, 'right': +1}
    
    def actions(self, state):
        index = state.index(' ')
        possible_moves = []
        if index // 3 > 0:
            possible_moves.append('up')
        if index // 3 < 2:
            possible_moves.append('down')
        if index % 3 > 0:
            possible_moves.append('left')
        if index % 3 < 2:
            possible_moves.append('right')
        return possible_moves
    
    def result(self, state, action):
        def swap(t, i, j):
            '''Auxiliary function for swapping two elements in a tuple.'''
            l = list(t)
            l[i], l[j] = l[j], l[i]
            return tuple(l)
        index = state.index(' ')
        return swap(state, index, index + self._action_values[action])
    
    def goal_test(self, state):
        return state == self._goal_state
    
    def step_cost(self, state, action):
        return 1

def _sliding_puzzle_3x3_visualizer(problem, state):
    '''Custom visualizer for the 3x3 sliding puzzle problem.'''
    for i in range(0, 9, 3):
        print(' ' + ' '.join(state[i:i + 3]) + ' ')

_visualizers[SlidingPuzzle3x3] = _sliding_puzzle_3x3_visualizer

In [9]:
problem = SlidingPuzzle3x3('12345678 ', '123 56478') #An example problem

In [11]:
from collections import deque
#Depth-first-search (tree version)
def DFS_Tree(problem, verbose = False):
  if problem.goal_test(problem.init_state): return solution(problem.init_state)
  frontier = deque([Node.root(problem.init_state)])
  if verbose: visualizer = Visualizer(problem)
  while frontier:
        if verbose: visualizer.visualize(frontier)
        node = frontier.popleft()
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            if problem.goal_test(child.state):
                return solution(child)
            frontier.appendleft(child)

#Depth-first-search (Graph version)
def DFS_Graph(problem, verbose=False):
  if problem.goal_test(problem.init_state): return solution(problem.init_state)
  frontier = deque([Node.root(problem.init_state)])
  explored = {problem.init_state}
  if verbose: visualizer = Visualizer(problem)
  while frontier:
      if verbose: visualizer.visualize(frontier)
      node = frontier.popleft()
      for action in problem.actions(node.state):
          child = Node.child(problem, node, action)
          if child.state not in explored:
              if problem.goal_test(child.state):
                  return solution(child)
              frontier.appendleft(child)
              explored.add(child.state)

#DFS_Tree(problem, verbose=True) #The tree goes into an infinite loop
DFS_Graph(problem, verbose=True)

Frontier at step 1

 1 2 3 
 4 5 6 
 7 8   
------------------------------------------------------------------------------------------------------------------------
Frontier at step 2

 1 2 3 
 4 5 6 
 7   8 

 1 2 3 
 4 5   
 7 8 6 
------------------------------------------------------------------------------------------------------------------------
Frontier at step 3

 1 2 3 
 4 5 6 
   7 8 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5   
 7 8 6 
------------------------------------------------------------------------------------------------------------------------


(['left', 'left', 'up'], 3)

In [12]:
#Depth-limited-search (Tree version)
def DLS_Tree(problem, limit, verbose=False):
    if problem.goal_test(problem.init_state): return solution(problem.init_state)
    frontier = deque([Node.root(problem.init_state)])
    depth =0
    if verbose: visualizer = Visualizer(problem)
    while frontier:
      if depth <= limit:
        if verbose: visualizer.visualize(frontier)
        node = frontier.popleft()
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            if problem.goal_test(child.state):
                return solution(child)
            frontier.appendleft(child)
            depth +=1
      else:
        return False

#Depth-limited-search (Tree version)
def DLS_Graph(problem, limit, verbose=False):
  if problem.goal_test(problem.init_state): return solution(problem.init_state)
  frontier = deque([Node.root(problem.init_state)])
  explored = {problem.init_state}
  depth = 0
  if verbose: visualizer = Visualizer(problem)
  while frontier:
    if depth <= limit:
      if verbose: visualizer.visualize(frontier)
      node = frontier.popleft()
      for action in problem.actions(node.state):
          child = Node.child(problem, node, action)
          if child.state not in explored:
              if problem.goal_test(child.state):
                  return solution(child), True
              frontier.appendleft(child)
              explored.add(child.state)
              depth +=1
    else:
      return False

#DLS_Tree(problem, 4,  verbose=True)
DLS_Graph(problem, 4, verbose=True)

Frontier at step 1

 1 2 3 
 4 5 6 
 7 8   
------------------------------------------------------------------------------------------------------------------------
Frontier at step 2

 1 2 3 
 4 5 6 
 7   8 

 1 2 3 
 4 5   
 7 8 6 
------------------------------------------------------------------------------------------------------------------------
Frontier at step 3

 1 2 3 
 4 5 6 
   7 8 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5   
 7 8 6 
------------------------------------------------------------------------------------------------------------------------


((['left', 'left', 'up'], 3), True)

In [13]:
#Iterative-deepening-search (Tree version)
def ITS_Tree(problem, verbose=False):
  max_depth=5
  for limit in range(max_depth):
    if(DLS_Tree(problem, limit, verbose=False)):
      return DLS_Tree(problem, limit, verbose=True)

  return False

#Iterative-deepening-search (Graph version)
def ITS_Graph(problem, verbose=False):
  max_depth=5
  for limit in range(0,max_depth):
    if (DLS_Graph(problem, limit, verbose=False)):
      return DLS_Graph(problem, limit, verbose=True)

  return False

ITS_Tree(problem, verbose=True)
#ITS_Graph(problem, verbose=True)

False